## Gianna's Exploration
### Determine Group Access Based On Usage Patterns

In [1]:
# For Downloading Data
!pip install googledrivedownloader

import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import sklearn
from zipfile import ZipFile
from google_drive_downloader import GoogleDriveDownloader as gdd
import datetime
from tqdm.notebook import tqdm

/opt/conda/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


#### Reading in google drive data

In [2]:
gdd.download_file_from_google_drive(file_id='1mIntdxoLWTbecLqILzMBQybsdNhjq3LM',
                                    dest_path='./data.zip',
                                    unzip=True)

In [3]:
#all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames = [i for i in glob.glob('./data/*')]

In [ ]:
%%time
csvs = [pd.read_csv(f) for f in all_filenames[:4]]
csvs_df = pd.concat(csvs); csvs_df.shape

In [ ]:
csvs_df.sample(5)

In [ ]:
# read in appclass mappings
app_2_class = pd.read_csv('app2appclass.csv')
app_2_class.head()

In [ ]:
app_2_class.appclass.nunique()

In [ ]:
app_2_class.appclass.value_counts()

In [ ]:
app_2_class_dict = app_2_class.set_index('application')['appclass'].to_dict()

In [ ]:
csvs_df['appclass'] = csvs_df['application'].map(app_2_class_dict)
csvs_df.sample(5)

In [ ]:
# Rename f0_ column to something readable
csvs_df.rename(columns={"f0_": "date"}, inplace = True)

In [ ]:
# filter for users that have 3 months of data at least ("its okay to drop those users")

In [ ]:
csvs_df.userid.nunique()

In [ ]:
csvs_df.shape[0]/csvs_df.userid.nunique()

In [ ]:
csvs_df['userid'].value_counts().hist(bins=25)

In [ ]:
csvs_df.head()

In [ ]:
date_min = csvs_df.date.min()
date_min

In [ ]:
date_max = csvs_df.date.max()
date_max

### Lets take a look at what we have in the first 4 files - containing a date range of '2020-09-02' to '2020-11-30'

In [ ]:
# Total user count per app class over whole time frame loaded in the file - > unique_user_count
# Average bussiness application a user acceses over the time frame of three months 
dl_up_counts_df = csvs_df.groupby('appclass')['download_tx_cnt','upload_tx_cnt'].sum()
dl_up_counts_df['unique_user_count'] = csvs_df.groupby('appclass')['userid'].nunique()

In [ ]:
# each classes download and upload count. Upload to download ratio
dl_up_counts_df['avg_dl_user'] = dl_up_counts_df['download_tx_cnt']/dl_up_counts_df['unique_user_count']
dl_up_counts_df['avg_ul_user'] = dl_up_counts_df['upload_tx_cnt']/dl_up_counts_df['unique_user_count']
dl_up_counts_df['ul_to_dl_ratio'] = dl_up_counts_df['upload_tx_cnt']/dl_up_counts_df['download_tx_cnt']

In [ ]:
# each classes percentage of upload and download traffic, plus total traffic.
dl_up_counts_df['percent_of_dl_traffic'] = dl_up_counts_df.groupby('appclass')['download_tx_cnt'].apply(lambda x : round((x / dl_up_counts_df['download_tx_cnt'].sum()) * 100,3))
dl_up_counts_df['percent_of_ul_traffic'] = dl_up_counts_df.groupby('appclass')['upload_tx_cnt'].apply(lambda x : round((x / dl_up_counts_df['upload_tx_cnt'].sum()) * 100,3))
dl_up_counts_df['percent_of_total_traffic'] = dl_up_counts_df.groupby('appclass')[['download_tx_cnt','upload_tx_cnt']].apply(lambda x : round((x['download_tx_cnt'][0] +  x['upload_tx_cnt'][0])/(dl_up_counts_df['download_tx_cnt'].sum()+dl_up_counts_df['upload_tx_cnt'].sum())*100,3))

In [ ]:
# of users that went to one of the app classes, how many unique applications did they go to?
unique_apps_df = csvs_df.groupby(['userid','appclass'])['application'].nunique().reset_index()
dl_up_counts_df['average_unique_apps_per_user'] = unique_apps_df[['appclass', 'application']].groupby('appclass').mean();

In [ ]:
# How many unique users visit each of the application clases
dl_up_counts_df['perct_users'] = round((dl_up_counts_df['unique_user_count']/csvs_df.userid.nunique()*100),3)

### What are we looking at:
*  "General Browsing" accounts for about 50% of all traffic. It has the most uploads and downlods by a lot.
*  "P2P", "Image Host", and "Tunnel" are very rare events.
*  Most users is "General Browsing", "Enterprise", "WebSearch", "Webmail, "Business", "IT Services"....All about 95% of users. There are some close runner ups too. 

In [ ]:
dl_up_counts_df.sort_values(by = 'percent_of_total_traffic', ascending = False)